# Testing for bias in model towards certain features

In [1]:
import onnx
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from onnxconverter_common import FloatTensorType
from skl2onnx import convert_sklearn
from src.tests.test_utils import *

### Load and train model

In [2]:
# specify existing model path
model_path = "../../../model/gboost.onnx"

In [3]:
df = add_checked(pd.read_csv("../../../data/synth_data_train_labeled.csv"))
X = df.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)
y = df['checked']

In [4]:
# # Uncomment to retrain model/make changes 
# 
# selector = VarianceThreshold()
# classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
# 
# pipeline = Pipeline(steps=[('feature selection', selector), ('classification', classifier)])
# 
# # Let's train a simple model
# pipeline.fit(X, y)
# 
# # Let's convert the model to ONNX
# onnx_model = convert_sklearn(
#     pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
#     target_opset=12)
# 
# sess = rt.InferenceSession(onnx_model.SerializeToString())
# 
# # Let's save the model
# onnx.save(onnx_model, model_path)

### Experiment_persoon_geslacht_vrouw (Whether the subject is male (0) or female (1))

In [5]:
data_path = '../../../data/Experiment_persoon_geslacht_vrouw/male_only.csv'
acc_male, acc_female, p_value, dataset_size, male_checked_cnt, female_checked_cnt = test_bias(data_path, model_path)

print(f'Accuracy for sample of {dataset_size} Men: {acc_male*100:.1f}%')
print(f'Accuracy for sample of {dataset_size} Women: {acc_female*100:.1f}%')
print(f'Percentage checked amongst 1000 men: {male_checked_cnt*100/dataset_size:.1f}%')
print(f'Percentage checked when gender is changed to female: {female_checked_cnt*100/dataset_size:.1f}%')

print(f"p-value: {p_value}")
print('---------------------------------------------------------------------------------')
if p_value < 0.05:
    print(f'Conclusion: Model showcases bias towards feature - persoon_geslacht_vrouw (gender)')
else:
    print(f'Conclusion: Model does not showcase bias towards feature - persoon_geslacht_vrouw (gender)')
print('---------------------------------------------------------------------------------')

Accuracy for sample of 1000 Men: 75.5%
Accuracy for sample of 1000 Women: 71.2%
Percentage checked amongst 1000 men: 34.4%
Percentage checked when gender is changed to female: 38.9%
p-value: 0.03678825564616459
---------------------------------------------------------------------------------
Conclusion: Model showcases bias towards feature - persoon_geslacht_vrouw (gender)
---------------------------------------------------------------------------------
